# Venmo Wrapped

Needs [Selenium](https://selenium-python.readthedocs.io/index.html).

In [ ]:
from datetime import datetime
import calendar
import itertools
import time
import os

from matplotlib import pyplot as plt
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

plt.rcParams['figure.figsize'] = [12, 8]

## 1. Getting the data

Since Venmo only allows you to download 60 days' worth of data at a time and I've had Venmo since 2015, I automated this part -- but if you haven't used Venmo for very long, it might be easier to just manually download the data.

In [ ]:
# Get start and end dates for each month
months = ['0'+str(x) if (len(str(x)) == 1) else x for x in range(1, 13)]
years = [str(x) for x in range(2015, 2021)]
month_years = list(itertools.product(*[months, years]))
date_ranges = [('{}-01-{}'.format(x[0], x[1]),
                '{}-{}-{}'.format(x[0], calendar.monthrange(int(x[1]), int(x[0]))[1], x[1]))
                for x in month_years]

Venmo doesn't allow the automated browser to log in, so first we spin up the browser:

In [ ]:
driver = webdriver.Chrome('/Users/lucy/opt/WebDriver/bin/chromedriver')
driver.get('https://venmo.com/account/sign-in')

Now **manually login to your Venmo account** in the automated browser window. 

Then run this:

In [ ]:
for date_range in date_ranges:
    print(date_range)
    driver.get('https://venmo.com/account/statement?end={}&start={}'.format(date_range[1], date_range[0]))

    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div/div[2]/div/div/div/div[1]/div/div[3]/a'))
    )
    element.click()
    
    time.sleep(5)
    
driver.close()

## 2. Analyzing the data

### 2.1 Loading & cleaning the data

In [ ]:
data_files = os.listdir('venmo_data/') # Assuming all your data is in this dir
data = pd.concat([pd.read_csv(os.path.join('venmo_data', x))[1:-1] for x in data_files])

In [ ]:
data = data.drop(axis='columns', labels=['Username', 'Amount (fee)', 'Funding Source',
                                         'Beginning Balance', 'Ending Balance',
                                         'Statement Period Venmo Fees', 'Terminal Location',
                                         'Destination', 'Year to Date Venmo Fees',
                                         'Disclaimer']).reset_index()
data.drop_duplicates(subset='ID', inplace=True) # Drop duplicate transactions

In [ ]:
# Converting "- $X.XX", "+ $X.XX" to floats
def convert_dollar_to_float(dollar):
    if dollar[0] == '+':
        return float(dollar.split('$')[-1].replace(',', ''))
    elif dollar[0] == '-':
        return float(dollar.split('$')[-1].replace(',', ''))*-1
    
data['Amount (total)'] = [convert_dollar_to_float(x) for x in data['Amount (total)']]

Venmo includes transactions like bank transfers in the data, but we are only interested in charges and payments:

In [ ]:
data = data[data['Type'].isin(['Charge', 'Payment'])]

Venmo also formats the statements as a:
- `Charge` if someone requested you and you completed the charge. In this case, the `To` field is your name and the `From` field is the person who charged you. 
- `Payment` if you paid someone. In this case, the `To` field is the name of the person you paid and the `From` field is your name.

In both cases, the sign of the `Amount (total)` field reveals whether it's an inflow or outflow. Since we're only interested in who we're paying vs. receiving money from, we need to condense the `To` and `From` field into one `person` field that gives the name of the other party involved in each transaction.

In [ ]:
data['person'] = [x.replace('Lucy Wu', '')
                      if x==x else x for x in data['From']+data['To']]

In [ ]:
outflows = data[data['Amount (total)'] < 0]
inflows = data[data['Amount (total)'] > 0]

In [ ]:
# Top 5 people paid the most $ to in 2020
outflows[outflows['Datetime'] > '2020-01-01'].groupby('person') \
    .sum().sort_values(by='Amount (total)').head(5)[['Amount (total)']]

In [ ]:
# Top 5 people paid most frequently in 2020
outflows[outflows['Datetime'] > '2020-01-01'].groupby('person').count() \
    .sort_values(by='Amount (total)', ascending=False).head(5)[['ID']] \
    .rename(columns={'ID': '# times'})

In [ ]:
# Top 5 people received the most $ from in 2020
inflows[inflows['Datetime'] > '2020-01-01'].groupby('person').sum() \
    .sort_values(by='Amount (total)', ascending=False).head(5)[['Amount (total)']]

In [ ]:
# Top 5 people received from most frequently in 2020
inflows[inflows['Datetime'] > '2020-01-01'].groupby('person').count() \
    .sort_values(by='Amount (total)', ascending=False).head(5)[['ID']] \
    .rename(columns={'ID': '# times'})

In [ ]:
# People paid the most $ to, all time
outflows.groupby('person').sum().sort_values(by='Amount (total)') \
    .head(5)[['Amount (total)']]

In [ ]:
# Top 5 people paid the most frequently, all time
outflows.groupby('person').count().sort_values(by='Amount (total)', ascending=False) \
    .head(5)[['ID']].rename(columns={'ID': '# times'})

In [ ]:
# Total outflows/inflows
print('Total outflows: ', round(outflows['Amount (total)'].sum(), 2))
print('Total inflows: +', round(inflows['Amount (total)'].sum(), 2), sep='')

In [ ]:
# Plot payments per month
outflows['month'] = [datetime.strftime(datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'), '%Y-%m')
                     for x in outflows['Datetime']]
inflows['month'] = [datetime.strftime(datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'), '%Y-%m')
                     for x in inflows['Datetime']]

outflows_monthly = outflows.groupby('month').sum()[['Amount (total)']].sort_index()
inflows_monthly = inflows.groupby('month').sum()[['Amount (total)']].sort_index()
data_monthly = outflows_monthly.join(inflows_monthly, how='outer',
                                     lsuffix='_out', rsuffix='_in').fillna(0)
data_monthly.rename(columns={'Amount (total)_out': 'outflows',
                             'Amount (total)_in': 'inflows'}, inplace=True)

In [ ]:
data_monthly.plot(color=['red', 'green'])
plt.title('Monthly inflows vs. outflows')
plt.ylabel('Amount USD')
plt.xlabel('')
plt.show()